# 04 - SCENARIO ENGINEERING

## [A] Objective

Paris goal = **deep cuts in agrofood emissions** to stay **well below 2°C** and ideally **1.5°C**. In our proposal, we’re doing that by **directly modifying feature values** in the CSV under:

* **Scenario A** → moderate reductions  
    * *Paris-aligned but moderate, nearer to “well below 2°C”*  
* **Scenario B** → aggressive, high-ambition reductions  
    * *More aggressive / 1.5°C-style ambition*  

Below are **columns that will be tuned** in `Agrofood_co2_emission_clean.csv`, and *how* they differ between A and B.

---

### 1. Fertilizer & Manure (big methane / N₂O levers)

These are explicitly mentioned in the proposal for Scenario A/B.

**Columns to tune**

* `Fertilizers Manufacturing`
* `Manure Management`
* `Manure applied to Soils`
* `Manure left on Pasture`

**Scenario A (moderate cuts)**

* Reduce each of these by, say, **10–30%** relative to baseline.
* Interpretation: better fertilizer efficiency, improved manure storage, partial adoption of low-emission practices.

**Scenario B (high ambition, 1.5°C-aligned)**

* Reduce by **40–70%+** where plausible.
* Interpretation: widespread precision fertilizer use, covered manure storage, anaerobic digesters, strong regulations on manure spreading.

### 2. On-farm Energy & Inputs

These hit CO₂ directly via fuel and electricity.

**Columns to tune**

* `On-farm Electricity Use`
* `On-farm energy use`
* `Pesticides Manufacturing`

**Scenario A**

* Reduce energy-use columns by **10–25%** (efficiency, some renewables).
* Slight reductions in `Pesticides Manufacturing` via integrated pest management.

**Scenario B**

* Strong reductions (**40–60%+**) in on-farm energy, assuming major shift to renewable power and electric machinery.
* Larger cut in `Pesticides Manufacturing` if we assume big adoption of low-input / regenerative practices.

### 3. Supply-Chain & Consumption Side (food system decarbonization)

These reflect processing, logistics, retail, and consumer behavior — critical for Paris alignment.

**Columns to tune**

* `Food Processing`
* `Food Packaging`
* `Food Transport`
* `Food Retail`
* `Food Household Consumption`
* `Agrifood Systems Waste Disposal`

**Scenario A**

* Moderate efficiency gains: reduce each by **10–20%**.
* Slight reduction in `Agrifood Systems Waste Disposal` (less waste, better landfill gas capture).

**Scenario B**

* Aggressive transformation: **30–60%** reduction in processing, packaging, transport and retail emissions.
* Strong drop in `Agrifood Systems Waste Disposal` (large food-waste cuts, circular economy, composting, biogas).

### 4. Land-Use & Fire Dynamics (key for Paris land-use pillar)

These affect **carbon sinks** and **land-use CO₂**, which are central in the Paris Agreement pathways.

**Columns to tune**

* `Forestland`  (often **negative** = carbon sink)
* `Net Forest conversion` (deforestation / land-use change)
* `Savanna fires`
* `Forest fires`
* `Drained organic soils (CO2)`
* `Fires in organic soils`
* `Fires in humid tropical forests`

**Scenario A**

* **Reduce fire and drainage emissions** by **10–25%**.
* Slightly **increase the carbon sink**: make `Forestland` more negative (e.g., +10–20% in magnitude).
* Mild reduction in `Net Forest conversion` (less deforestation).

**Scenario B**

* **Large reductions** in all fire-related and drainage emissions (**40–70%+**).
* Much stronger expansion of the sink: `Forestland` becomes substantially more negative (more re/afforestation).
* `Net Forest conversion` pushed close to zero or strongly reduced (halt deforestation, net reforestation).

### 5. What features are *not* directly tuned

* `total_emission` – this is the **target** the model predicts.
* `Average Temperature °C` – this is an **outcome** we want to influence via emissions, not an input lever.
* `Area`, `Year` – identifiers / time index, not policy variables.
* `Rural population`, `Urban population` - demographic breakdowns that don’t directly affect emissions.
* `Total Population - Male`, `Total Population - Female` – demographic details not directly tied to emissions.

---

## [B] Exact multipliers per column

### Land-use & fire emissions

| Column                                | Scenario A | Scenario B |
| ------------------------------------- | ---------: | ---------: |
| `Savanna fires`                       |       0.80 |       0.40 |
| `Forest fires`                        |       0.80 |       0.40 |
| `Fires in organic soils`              |       0.80 |       0.40 |
| `Fires in humid tropical forests`     |       0.80 |       0.40 |
| `Drained organic soils (CO2)`         |       0.80 |       0.40 |
| `Net Forest conversion`               |       0.80 |       0.30 |
| `Forestland` *(sink, often negative)* |   **1.20** |   **1.50** |
  
> For `Forestland`, multiplying by >1 makes the **sink stronger** (more negative if values are negative).  

### Fertilizer, manure, rice & residues

| Column                      | Scenario A | Scenario B |
| --------------------------- | ---------: | ---------: |
| `Rice Cultivation`          |       0.80 |       0.50 |
| `Fertilizers Manufacturing` |       0.80 |       0.50 |
| `Manure Management`         |       0.80 |       0.50 |
| `Manure applied to Soils`   |       0.80 |       0.50 |
| `Manure left on Pasture`    |       0.85 |       0.60 |
| `Crop Residues`             |       0.90 |       0.60 |
| `IPPU`                      |       0.90 |       0.70 |

### Energy & supply-chain / consumption

| Column                            | Scenario A | Scenario B |
| --------------------------------- | ---------: | ---------: |
| `On-farm Electricity Use`         |       0.85 |       0.50 |
| `On-farm energy use`              |       0.85 |       0.50 |
| `Pesticides Manufacturing`        |       0.85 |       0.60 |
| `Food Processing`                 |       0.90 |       0.60 |
| `Food Packaging`                  |       0.85 |       0.60 |
| `Food Transport`                  |       0.85 |       0.50 |
| `Food Retail`                     |       0.90 |       0.70 |
| `Food Household Consumption`      |       0.90 |       0.70 |
| `Agrifood Systems Waste Disposal` |       0.80 |       0.30 |

---

## [C] Python function to generate Scenario A & B DataFrames

Helper function to create scenario DataFrames based on the above multipliers

In [1]:
import pandas as pd

# Define multipliers per scenario
SCENARIO_MULTIPLIERS = {
    "A": {
        # Land-use & fire
        "Savanna fires": 0.80,
        "Forest fires": 0.80,
        "Fires in organic soils": 0.80,
        "Fires in humid tropical forests": 0.80,
        "Drained organic soils (CO2)": 0.80,
        "Net Forest conversion": 0.80,
        # Forest sink (typically negative -> make more negative)
        "Forestland": 1.20,

        # Fertilizer / manure / rice / residues
        "Rice Cultivation": 0.80,
        "Fertilizers Manufacturing": 0.80,
        "Manure Management": 0.80,
        "Manure applied to Soils": 0.80,
        "Manure left on Pasture": 0.85,
        "Crop Residues": 0.90,
        "IPPU": 0.90,

        # Energy & supply chain
        "On-farm Electricity Use": 0.85,
        "On-farm energy use": 0.85,
        "Pesticides Manufacturing": 0.85,
        "Food Processing": 0.90,
        "Food Packaging": 0.85,
        "Food Transport": 0.85,
        "Food Retail": 0.90,
        "Food Household Consumption": 0.90,
        "Agrifood Systems Waste Disposal": 0.80,
    },
    "B": {
        # Land-use & fire
        "Savanna fires": 0.40,
        "Forest fires": 0.40,
        "Fires in organic soils": 0.40,
        "Fires in humid tropical forests": 0.40,
        "Drained organic soils (CO2)": 0.40,
        "Net Forest conversion": 0.30,
        # Stronger forest sink
        "Forestland": 1.50,

        # Fertilizer / manure / rice / residues
        "Rice Cultivation": 0.50,
        "Fertilizers Manufacturing": 0.50,
        "Manure Management": 0.50,
        "Manure applied to Soils": 0.50,
        "Manure left on Pasture": 0.60,
        "Crop Residues": 0.60,
        "IPPU": 0.70,

        # Energy & supply chain
        "On-farm Electricity Use": 0.50,
        "On-farm energy use": 0.50,
        "Pesticides Manufacturing": 0.60,
        "Food Processing": 0.60,
        "Food Packaging": 0.60,
        "Food Transport": 0.50,
        "Food Retail": 0.70,
        "Food Household Consumption": 0.70,
        "Agrifood Systems Waste Disposal": 0.30,
    },
}


def apply_scenario(df: pd.DataFrame, scenario: str) -> pd.DataFrame:
    """
    Return a new dataframe with emissions drivers adjusted
    according to Scenario A or B multipliers.
    """
    if scenario not in SCENARIO_MULTIPLIERS:
        raise ValueError(f"Unknown scenario '{scenario}'. Use 'A' or 'B'.")

    df_new = df.copy()
    multipliers = SCENARIO_MULTIPLIERS[scenario]

    for col, factor in multipliers.items():
        if col in df_new.columns:
            df_new[col] = df_new[col] * factor
        else:
            # Optional: warn if a configured column is missing
            print(f"Warning: column '{col}' not in dataframe, skipping.")
            pass

    df_new["scenario"] = scenario
    return df_new


def create_scenario_dfs(df: pd.DataFrame):
    """
    Convenience helper to generate both Scenario A and B
    versions of the baseline dataframe.
    """
    df_A = apply_scenario(df, "A")
    df_B = apply_scenario(df, "B")
    return df_A, df_B


Running helper function to create scenario DataFrames

In [ ]:
df_base = pd.read_csv("Agrofood_co2_emission_clean.csv")
df_A, df_B = create_scenario_dfs(df_base)


Optional: Export DataFrames to CSVs

In [ ]:
# df_A.to_csv("Agrofood_co2_emission_clean_scenarioA.csv", index=False)
df_A.head()

,Area,Year,Savanna fires,Forest fires,Crop Residues,Rice Cultivation,Drained organic soils (CO2),Pesticides Manufacturing,Food Transport,Forestland,...,Fires in organic soils,Fires in humid tropical forests,On-farm energy use,Rural population,Urban population,Total Population - Male,Total Population - Female,total_emission,Average Temperature °C,scenario
0,Afghanistan,1990,11.77896,0.04456,185.04693,548.800,0.0,10.036361,53.647920,-2866.5636,...,0.0,0.0,115.03254,9655167.0,2593947.0,5348387.0,5346409.0,2198.963539,0.536167,A
1,Afghanistan,1991,11.77896,0.04456,188.54739,542.528,0.0,9.955262,52.030625,-2866.5636,...,0.0,0.0,115.03254,10230490.0,2763167.0,5372959.0,5372208.0,2323.876629,0.020667,A
2,Afghanistan,1992,11.77896,0.04456,176.88069,548.800,0.0,9.955262,45.319450,-2866.5636,...,0.0,0.0,115.03254,10995568.0,2985663.0,6028494.0,6028939.0,2356.304229,-0.259583,A
3,Afghanistan,1993,11.77896,0.04456,207.73575,548.800,0.0,9.955262,46.207445,-2866.5636,...,0.0,0.0,115.03254,11858090.0,3237009.0,7003641.0,7000119.0,2368.470529,0.101917,A
4,Afghanistan,1994,11.77896,0.04456,217.84446,564.480,0.0,9.955262,45.889290,-2866.5636,...,0.0,0.0,115.03254,12690115.0,3482604.0,7733458.0,7722096.0,2500.768729,0.372250,A


In [ ]:
# df_B.to_csv("Agrofood_co2_emission_clean_scenarioB.csv", index=False)
df_B.head()

,Area,Year,Savanna fires,Forest fires,Crop Residues,Rice Cultivation,Drained organic soils (CO2),Pesticides Manufacturing,Food Transport,Forestland,...,Fires in organic soils,Fires in humid tropical forests,On-farm energy use,Rural population,Urban population,Total Population - Male,Total Population - Female,total_emission,Average Temperature °C,scenario
0,Afghanistan,1990,5.88948,0.02228,123.36462,343.00,0.0,7.084490,31.55760,-3583.2045,...,0.0,0.0,67.6662,9655167.0,2593947.0,5348387.0,5346409.0,2198.963539,0.536167,B
1,Afghanistan,1991,5.88948,0.02228,125.69826,339.08,0.0,7.027244,30.60625,-3583.2045,...,0.0,0.0,67.6662,10230490.0,2763167.0,5372959.0,5372208.0,2323.876629,0.020667,B
2,Afghanistan,1992,5.88948,0.02228,117.92046,343.00,0.0,7.027244,26.65850,-3583.2045,...,0.0,0.0,67.6662,10995568.0,2985663.0,6028494.0,6028939.0,2356.304229,-0.259583,B
3,Afghanistan,1993,5.88948,0.02228,138.49050,343.00,0.0,7.027244,27.18085,-3583.2045,...,0.0,0.0,67.6662,11858090.0,3237009.0,7003641.0,7000119.0,2368.470529,0.101917,B
4,Afghanistan,1994,5.88948,0.02228,145.22964,352.80,0.0,7.027244,26.99370,-3583.2045,...,0.0,0.0,67.6662,12690115.0,3482604.0,7733458.0,7722096.0,2500.768729,0.372250,B
